In [ ]:
! pip install spatialdata


In [8]:
from scipy.datasets import face
import spatialdata
import pytest
from spatialdata.models import Image2DModel

# raw data
image = face()
print(type(image))
image.shape

<class 'numpy.ndarray'>


(768, 1024, 3)

In [13]:

# numpy arrays do not pass validation
with pytest.raises(ValueError, match="Unsupported data type: <class 'numpy.ndarray'>."):
    Image2DModel().validate(image)

# let's parse the data
parsed_image = Image2DModel.parse(image, dims=("y", "x", "c"))

# now passes validation (=can be placed inside a SpatialData object)
Image2DModel().validate(parsed_image)

# parsed_image is a regular spatial_image.SpatialImage object (discussed later), and as such it can be used outside
# the SpatialData library (e.g. with the libraries dask-image, xarray-spatial, ...)
parsed_image

INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           


<xarray.SpatialImage 'image' (c: 3, y: 768, x: 1024)>
dask.array<transpose, shape=(3, 768, 1024), dtype=uint8, chunksize=(3, 768, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 0 1 2
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 763.5 764.5 765.5 766.5 767.5
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 1.022e+03 1.022e+03 1.024e+03
Attributes:
    transform:  {'global': Identity }

In [14]:
from spatialdata.transformations import Scale, get_transformation, remove_transformation

print(type(image))
parsed_image = Image2DModel.parse(image, dims=("y", "x", "c"))

# the default transformation is an identity
get_transformation(parsed_image)

# let's add custom transformations, first let's remove it
remove_transformation(parsed_image)

# to add new ones let's re-parse parse_image; note that the type of parsed_image is now a SpatialImage, not a numpy.array as before.
# This is not a problem as the single-dispath construct takes care of handling the different data types.
# No need to pass dims now, as the dims are already specified in the parsed_image object
parsed_image = Image2DModel.parse(
    parsed_image,
    transformations={"scale_space": Scale([2.0], axes=("x",)), "another_space": Scale([2.0, 3.0], axes=("y", "x"))},
)
print(type(parsed_image))

# let's check that the axes are correct
from spatialdata.models import get_axes_names

get_axes_names(parsed_image)

<class 'numpy.ndarray'>
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
<class 'spatial_image.SpatialImage'>


('c', 'y', 'x')

In [15]:
# when the input data has not dimensions equal to ('c', 'y', 'x'), we can use the dims argument
# of .parse() to transpose the data

# image is ('y', 'x', 'c')
Image2DModel.parse(image, dims=("y", "x", "c"))

INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           


<xarray.SpatialImage 'image' (c: 3, y: 768, x: 1024)>
dask.array<transpose, shape=(3, 768, 1024), dtype=uint8, chunksize=(3, 768, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 0 1 2
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 763.5 764.5 765.5 766.5 767.5
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 1.022e+03 1.022e+03 1.024e+03
Attributes:
    transform:  {'global': Identity }

In [16]:
# types supported by the single-dispatch construct of the parser

# numpy array (we saw in examples before)
print(type(image))

# xarray data arrays (we saw it in an example before). Remember that spatial_image.SpatialImage is a subclass of xarray.DataArray
print(type(parsed_image))

# dask ("lazy") arrays
from dask.array import from_array

lazy_array = from_array(image)
print(type(lazy_array))
Image2DModel.parse(lazy_array, dims=("y", "x", "c"))

<class 'numpy.ndarray'>
<class 'spatial_image.SpatialImage'>
<class 'dask.array.core.Array'>
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           


<xarray.SpatialImage 'image' (c: 3, y: 768, x: 1024)>
dask.array<transpose, shape=(3, 768, 1024), dtype=uint8, chunksize=(3, 768, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 0 1 2
  * y        (y) float64 0.5 1.5 2.5 3.5 4.5 ... 763.5 764.5 765.5 766.5 767.5
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 1.022e+03 1.022e+03 1.024e+03
Attributes:
    transform:  {'global': Identity }

In [17]:
# let's access the c coords directly using the xarray syntax
print(parsed.c.values)

NameError: name 'parsed' is not defined